# Description
This notebook fits the cultivation model by maximum-likelihood.

It does not create plots, but saves the MLE as a JSON file to the `/processed` directory.

In [1]:
import json
import numpy
import pandas
import pathlib
import scipy.optimize
import sys

import calibr8
import murefi

import models

DP_PROCESSED = pathlib.Path("processed")

In [2]:
cm_biomass = models.get_biomass_model()
cm_glucose = models.get_glucose_model()
model = models.MonodModel()
dataset = murefi.load_dataset(DP_PROCESSED / "cultivation_dataset.h5")
theta_mapping = models.get_parameter_mapping()
theta_mapping.as_dataframe()

,S0,X0,mu_max,K_S,Y_XS
rid,,,,,
A02,S0,X0_A02,mu_max,K_S,Y_XS
A03,S0,X0_A03,mu_max,K_S,Y_XS
A04,S0,X0_A04,mu_max,K_S,Y_XS
A05,S0,X0_A05,mu_max,K_S,Y_XS
A06,S0,X0_A06,mu_max,K_S,Y_XS
A07,S0,X0_A07,mu_max,K_S,Y_XS
A08,S0,X0_A08,mu_max,K_S,Y_XS
B02,S0,X0_B02,mu_max,K_S,Y_XS
B03,S0,X0_B03,mu_max,K_S,Y_XS


### Fit the model by MLE

In [3]:
# Create the objective function
objective = murefi.objectives.for_dataset(
    dataset=dataset,
    model=model,
    parameter_mapping=theta_mapping,
    calibration_models=[cm_glucose, cm_biomass],
)

# If the guess has a NaN likelihood there's a problem.
ll_guess = objective(theta_mapping.guesses)
if not numpy.isfinite(ll_guess):
    raise ValueError(f"Your guess is shit. objective(guess)={ll_guess}")

# Find the maximum with scipy
fit_result = scipy.optimize.minimize(
    objective,
    x0=theta_mapping.guesses,
    bounds=theta_mapping.bounds,
    # The callback gives us a bit of a progress bar...
    callback=lambda x: sys.stdout.write("."),
)
if calibr8.optimization._warn_hit_bounds(
    fit_result.x, theta_mapping.bounds, theta_mapping.theta_names
):
    display(fit_result)
print()
for tf, tname in zip(fit_result.x, theta_mapping.theta_names):
    print(f"{tname: <10}{tf}")
print()
print(fit_result)

.....................

C:\Users\osthege\AppData\Local\Continuum\miniconda3\envs\murefi_env\lib\site-packages\scipy\optimize\_numdiff.py:497: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


.
K_S       0.0010045218851312565
S0        17.0488605437842
X0_A02    0.24617364802039868
X0_A03    0.26520831557943336
X0_A04    0.2793818413567048
X0_A05    0.26624865198211256
X0_A06    0.2491969485474249
X0_A07    0.25183997665001956
X0_A08    0.2406264308804667
X0_B02    0.26193231893458785
X0_B03    0.2695816261748549
X0_B04    0.28559264429842524
X0_B05    0.25273862550600923
X0_B06    0.2515303987238219
X0_B07    0.2552795302923193
X0_B08    0.2393141119333401
X0_C02    0.2629214812502805
X0_C03    0.2704096517373555
X0_C04    0.2754482769467562
X0_C05    0.251039664838805
X0_C06    0.2566625262251159
X0_C07    0.23964542398136812
X0_C08    0.24032155093845772
X0_D02    0.2638686533791734
X0_D03    0.2761059491529098
X0_D04    0.2809656206526789
X0_D05    0.25667005244231944
X0_D06    0.24807711807217417
X0_D07    0.24128310621792476
X0_D08    0.2490539453514991
Y_XS      0.6674710079620007
mu_max    0.4246050447438685

      fun: -881.4197086567664
 hess_inv: <32x32 LbfgsInvH

In [4]:
with open(DP_PROCESSED / "full_dataset_mle.json", "w") as jfile:
    json.dump(
        {k: v for k, v in zip(theta_mapping.parameters.keys(), fit_result.x)},
        jfile,
        indent=4,
    )

In [5]:
%load_ext watermark
%watermark

Last updated: 2021-12-15T14:53:48.623332+01:00

Python implementation: CPython
Python version       : 3.7.9
IPython version      : 7.24.1

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 158 Stepping 10, GenuineIntel
CPU cores   : 6
Architecture: 64bit

